### 1. Import

In [1]:
cd c:\Users\nlinl\Desktop\Important\EECS6892\project\soccer_twos

c:\Users\nlinl\Desktop\Important\EECS6892\project\soccer_twos


c:\Users\nlinl\anaconda3\envs\rl\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# import packages
import time
import os
import torch
from torch import nn
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
from mlagents_envs.environment import UnityEnvironment as UE

In [3]:
# import modules
from environment import SoccerTwosEnvWrapper
from agents.dqn import DeepQN
from agents.jal import JointActionLearner
from models.models import *

In [4]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

### 2. Initialization

In [5]:
# in case set up fails
try:
    ue.close()
except:
    pass

In [6]:
# start up environment
root = r'C:/Users/nlinl/Desktop/Important/EECS6892/project/ml-agents/training-envs-executables'
ue = UE(file_name = root + '/' +'SoccerTwos.exe')
env = SoccerTwosEnvWrapper(ue)

### 3. Matchup

In [7]:
TEST_EPISODES = 100
episode_rewards = []
num_agents = env.num_agents

path_to_team1 = './weights/dqn_best_3000.pt'
path_to_team2 = './weights/jal_last.pt'

agent1  = DeepQN
agent2 = JointActionLearner
 
    
num_agents = env.num_agents

dqn_agents = [agent1(
    model = LinearModel,
    in_size = env.obssize,
    out_size = env.actsize
) for _ in range(2)]

for agent in dqn_agents:
    agent.model.load_state_dict(torch.load(path_to_team1))


jal_agents = [agent2(
    agent_id = a,
    num_agents_per_team = 2,
    teammate_idx = a + (-1)**(a%2),
    model = LinearModel,
    obs_size = env.obssize,
    act_size = env.actsize
) for a in range(2)]

for agent in jal_agents:
    agent.model.load_state_dict(torch.load(path_to_team2))

agents = dqn_agents + jal_agents
# dqn: purple, jal: blue

for episode in range(TEST_EPISODES):

    states = np.array(env.reset())
    done = False
    episode_reward = [0]*num_agents

    while not done:
    
        # choosing action
        actions = [None] * num_agents

        for a in range(num_agents):

            # get agent
            agent = agents[a]
            
            state_t = torch.tensor(states[a])
            state_t = torch.unsqueeze(state_t, 0)
            action = agent.sample_action_from_policy(state_t)

            actions[a] = action

        # set action in environment
        for a, action in enumerate(actions):
            
            env.set_action(a, action)

        # take step in environment
        next_states, rewards, dones = env.step()
        
        # update rewards
        for a in range(num_agents):
            episode_reward[a] += rewards[a]

        # state transition 
        states = next_states
        done = np.all(dones)

    episode_rewards.append(episode_reward)
    
    # print progress
    for a, agent in enumerate(agents):
        agent.print_testing_progrees(a, episode, episode_reward[a])
    print( '#'*40)


env.close()

# plot test rewards
episode_rewards = np.array(episode_rewards).T
plt.plot(np.arange(1, TEST_EPISODES+1, 1), episode_rewards[0], label = 'dqn')
plt.plot(np.arange(1, TEST_EPISODES+1, 1), episode_rewards[-1], label = 'jal')
plt.title('episode rewards against episode count')
plt.xlabel('episode')
plt.ylabel('reward')
plt.legend()
plt.show()
plt.savefig('./plots/matchup_plot.png')
print('plot saved at /plots/matchup_plot.png') 

print(f'total rewards: dqn = {episode_rewards[0].sum()} vs. jal = {episode_rewards[-1].sum()}')

AGENT 0, EPISODE 0: reward 0.98
AGENT 1, EPISODE 0: reward 0.98
AGENT 2, EPISODE 0: reward -1.00
AGENT 3, EPISODE 0: reward -1.00
########################################
AGENT 0, EPISODE 1: reward -1.00
AGENT 1, EPISODE 1: reward -1.00
AGENT 2, EPISODE 1: reward 0.91
AGENT 3, EPISODE 1: reward 0.91
########################################
AGENT 0, EPISODE 2: reward 0.79
AGENT 1, EPISODE 2: reward 0.79
AGENT 2, EPISODE 2: reward -1.00
AGENT 3, EPISODE 2: reward -1.00
########################################
AGENT 0, EPISODE 3: reward 0.74
AGENT 1, EPISODE 3: reward 0.74
AGENT 2, EPISODE 3: reward -1.00
AGENT 3, EPISODE 3: reward -1.00
########################################
AGENT 0, EPISODE 4: reward -1.00
AGENT 1, EPISODE 4: reward -1.00
AGENT 2, EPISODE 4: reward 0.74
AGENT 3, EPISODE 4: reward 0.74
########################################
AGENT 0, EPISODE 5: reward 0.82
AGENT 1, EPISODE 5: reward 0.82
AGENT 2, EPISODE 5: reward -1.00
AGENT 3, EPISODE 5: reward -1.00
###############

KeyboardInterrupt: 